In [1]:
import sys
sys.path.append('..')
import os, glob, cv2, time
import numpy as np
import tensorflow as tf

from utils import *
from scipy import io as mat
from matplotlib import pyplot as plt

tf.set_random_seed(777)

In [2]:
tmp = np.load('../data/input/data_2d.npz')
img = tmp['img']
img = np.reshape(img, (img.shape[0], img.shape[1], img.shape[2], 1))
lab = np.ceil(tmp['lab'])
lab = np.reshape(lab, (lab.shape[0], lab.shape[1], lab.shape[2], 1))

In [3]:
tum_in_img = []
tum_in_lab = []

tum_no_img = []
tum_no_lab = []
count = 0
for idx, i in enumerate(lab):
    sum_ = np.sum(i)
    if sum_ != 0:
        tum_in_img.append(img[idx])
        tum_in_lab.append(i)
    else : 
        tum_no_img.append(img[idx])
        tum_no_lab.append(i)
        
tum_in_img = np.array(tum_in_img)
tum_in_lab = np.array(tum_in_lab)
tum_no_img = np.array(tum_no_img)
tum_no_lab = np.array(tum_no_lab)

In [4]:
print(tum_in_img.shape)
print(tum_in_lab.shape)
print(tum_no_img.shape)
print(tum_no_lab.shape)

del img, lab, tmp

(633, 256, 256, 1)
(633, 256, 256, 1)
(2955, 256, 256, 1)
(2955, 256, 256, 1)


In [5]:
X = tf.placeholder(tf.float32, shape=[None, 256, 256, 1])
Y = tf.placeholder(tf.float32, shape=[None, 256, 256, 1])

weight = {
    'we1_1':init_w('we1_1', [3, 3, 1, 64]),
    'we1_2':init_w('we1_2', [3, 3, 64, 64]),
    'we2_1':init_w('we2_1', [3, 3, 64, 128]),
    'we2_2':init_w('we2_2', [3, 3, 128, 128]),
    'we3_1':init_w('we3_1', [3, 3, 128, 256]),
    'we3_2':init_w('we3_2', [3, 3, 256, 256]),
    'we4_1':init_w('we4_1', [3, 3, 256, 512]),
    'we4_2':init_w('we4_2', [3, 3, 512, 512]),
    'we5_1':init_w('we5_1', [3, 3, 512, 1024]),
    'we5_2':init_w('we5_2', [3, 3, 1024, 1024]),
    
    'wd4_3':init_w('wd4_3', [4, 4, 1024, 1024]),
    'wd4_2':init_w('wd4_2', [3, 3, 1536, 512]),
    'wd4_1':init_w('wd4_1', [3, 3, 512, 512]),
    'wd3_3':init_w('wd3_3', [4, 4, 512, 512]),
    'wd3_2':init_w('wd3_2', [3, 3, 768, 256]),
    'wd3_1':init_w('wd3_1', [3, 3, 256, 256]),
    'wd2_3':init_w('wd2_3', [4, 4, 256, 256]),
    'wd2_2':init_w('wd2_2', [3, 3, 384, 128]),
    'wd2_1':init_w('wd2_1', [3, 3, 128, 128]),
    'wd1_3':init_w('wd1_3', [4, 4, 128, 128]),
    'wd1_2':init_w('wd1_2', [3, 3, 192, 64]),
    'wd1_1':init_w('wd1_1', [3, 3, 64, 64]),
    
    'wout':init_w('wout', [1,1,64,1])
}

In [6]:
def model(X, w):
    
    en1 = tf.nn.relu(conv2d(X, w['we1_1']))
    en1 = tf.nn.relu(conv2d(en1, w['we1_2']))
    
    en2 = max_pool_2d(en1)
    en2 = tf.nn.relu(conv2d(en2, w['we2_1']))
    en2 = tf.nn.relu(conv2d(en2, w['we2_2']))
    
    en3 = max_pool_2d(en2)
    en3 = tf.nn.relu(conv2d(en3, w['we3_1']))
    en3 = tf.nn.relu(conv2d(en3, w['we3_2']))
    
    en4 = max_pool_2d(en3)
    en4 = tf.nn.relu(conv2d(en4, w['we4_1']))
    en4 = tf.nn.relu(conv2d(en4, w['we4_2']))
    
    en5 = max_pool_2d(en4)
    en5 = tf.nn.relu(conv2d(en5, w['we5_1']))
    en5 = tf.nn.relu(conv2d(en5, w['we5_2']))
    
    de4 = deconv2d(en5, w['wd4_3'], (tf.shape(X)[0],32,32,1024),
                   strides=[1,2,2,1])
    de4 = tf.concat((en4, de4), axis = 3)
    de4 = tf.nn.relu(conv2d(de4, w['wd4_2']))
    de4 = tf.nn.relu(conv2d(de4, w['wd4_1']))

    de3 = deconv2d(de4, w['wd3_3'], (tf.shape(X)[0],64,64,512),
                   strides=[1,2,2,1])
    de3 = tf.concat((en3, de3), axis = 3)
    de3 = tf.nn.relu(conv2d(de3, w['wd3_2']))
    de3 = tf.nn.relu(conv2d(de3, w['wd3_1']))
    
    de2 = deconv2d(de3, w['wd2_3'], (tf.shape(X)[0],128,128,256),
                   strides=[1,2,2,1])
    de2 = tf.concat((en2, de2), axis = 3)
    de2 = tf.nn.relu(conv2d(de2, w['wd2_2']))
    de2 = tf.nn.relu(conv2d(de2, w['wd2_1']))
    
    de1 = deconv2d(de2, w['wd1_3'], (tf.shape(X)[0],256,256,128),
                   strides=[1,2,2,1])
    de1 = tf.concat((en1, de1), axis = 3)
    de1 = tf.nn.relu(conv2d(de1, w['wd1_2']))
    de1 = tf.nn.relu(conv2d(de1, w['wd1_1']))
    
    out = conv2d(de1, w['wout'])
    
    output = {
        
        'out':out,
        'en1':en1,
        'en2':en2,
        'en3':en3,
        'en4':en4,
        'en5':en5,
        'de4':de4,
        'de3':de3,
        'de2':de2,
        'de1':de1,
    }
    
    return output

def dice_coefficient(label, infer):
    
    label_flatten = tf.layers.flatten(label)
    infer_flatten = tf.layers.flatten(infer)
    TP = tf.reduce_sum(label_flatten*infer_flatten) + 1e-7
    FN = tf.reduce_sum(label_flatten*(1-infer_flatten)) + 1e-7
    FP = tf.reduce_sum((1-label_flatten)*infer_flatten) + 1e-7
    
    bottom = FN+FP+(2*TP) + 1e-7
    up = 2*TP + 1e-7
    return up/bottom + 1e-7

def make_batch(batch_size):
    thr = np.uint8(batch_size/2)
    tum_seq = np.random.choice(len(tum_in_img), thr, replace=False)
    no_seq = np.random.choice(len(tum_no_img), batch_size-thr, replace=False)
    
    batch_x = tum_in_img[tum_seq]
    batch_y = tum_in_lab[tum_seq]
    
    batch_x = np.concatenate((batch_x, tum_no_img[no_seq]), axis = 0)
    batch_y = np.concatenate((batch_y, tum_no_lab[no_seq]), axis = 0)
    
    return batch_x, batch_y

In [7]:
with tf.device('/gpu:1'):
    pred = model(X, weight)
    coeff = dice_coefficient(Y, tf.nn.sigmoid(pred['out']))
    cost = -coeff
with tf.device('/gpu:0'):
    train = tf.train.AdamOptimizer(1e-6).minimize(cost)

In [10]:
val_coeff=[]
val_cost=[]
#gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.9)
#with tf.Session(config=tf.ConfigProto(log_device_placement=True, gpu_options=gpu_options)) as sess:
with tf.Session() as sess:

    sess.run(tf.global_variables_initializer())
    saver = tf.train.Saver()

    start_time = time.time()

    epochs = 1000

    batch = 20

    total_batch = int(2*len(tum_no_img)/batch)

    for epoch in range(epochs):

        for i in range(2*total_batch):
            
            batch_x, batch_y = make_batch(batch)
            
            batch_idx = np.random.choice(batch, batch, replace=False)            
            batch_x = batch_x[batch_idx]
            batch_y = batch_y[batch_idx]
            sess.run(train, feed_dict={X:batch_x, Y:batch_y})
            val_coeff.append(sess.run(coeff, feed_dict={X:batch_x, Y:batch_y}))
            val_cost.append(sess.run(cost, feed_dict={X:batch_x, Y:batch_y}))
                             
            if (i+1)%30 == 0:

                print("%d/%d"%(epoch+1, epochs))

                print("%d/%d"%(i+1, total_batch))

                print("Train cost : ", sess.run(cost, 
                                                feed_dict={X:batch_x, Y:batch_y}))
                print("Train coef : ", sess.run(coeff, 
                                                feed_dict={X:batch_x, Y:batch_y}))

                print('-----------------')

        if (epoch+1)%100 == 0:
            saver.save(sess, './checkpoint/%d'%(epoch+1))
    print('Time : ', time.time()-start_time)            

1/1000
30/147
Train cost :  -0.00558408
Train coef :  0.00558408
-----------------
1/1000
60/147
Train cost :  -0.0045481
Train coef :  0.0045481
-----------------
1/1000
90/147
Train cost :  -0.00559351
Train coef :  0.00559351
-----------------
1/1000
120/147
Train cost :  -0.00413415
Train coef :  0.00413415
-----------------
2/1000
30/147
Train cost :  -0.00413819
Train coef :  0.00413819
-----------------
2/1000
60/147
Train cost :  -0.00567757
Train coef :  0.00567757
-----------------
2/1000
90/147
Train cost :  -0.00348242
Train coef :  0.00348242
-----------------
2/1000
120/147
Train cost :  -0.00588985
Train coef :  0.00588985
-----------------
3/1000
30/147
Train cost :  -0.00541093
Train coef :  0.00541093
-----------------
3/1000
60/147
Train cost :  -0.00527154
Train coef :  0.00527154
-----------------
3/1000
90/147
Train cost :  -0.00485212
Train coef :  0.00485212
-----------------
3/1000
120/147
Train cost :  -0.00384785
Train coef :  0.00384785
-----------------
4/1

26/1000
60/147
Train cost :  -0.175137
Train coef :  0.175137
-----------------
26/1000
90/147
Train cost :  -0.17984
Train coef :  0.17984
-----------------
26/1000
120/147
Train cost :  -0.378932
Train coef :  0.378932
-----------------
27/1000
30/147
Train cost :  -0.196048
Train coef :  0.196048
-----------------
27/1000
60/147
Train cost :  -0.340525
Train coef :  0.340525
-----------------
27/1000
90/147
Train cost :  -0.45994
Train coef :  0.45994
-----------------
27/1000
120/147
Train cost :  -0.123973
Train coef :  0.123973
-----------------
28/1000
30/147
Train cost :  -0.436355
Train coef :  0.436355
-----------------
28/1000
60/147
Train cost :  -0.47037
Train coef :  0.47037
-----------------
28/1000
90/147
Train cost :  -0.118688
Train coef :  0.118688
-----------------
28/1000
120/147
Train cost :  -0.450716
Train coef :  0.450716
-----------------
29/1000
30/147
Train cost :  -0.316722
Train coef :  0.316722
-----------------
29/1000
60/147
Train cost :  -0.238237
Trai

Train coef :  0.633018
-----------------
52/1000
30/147
Train cost :  -0.482054
Train coef :  0.482054
-----------------
52/1000
60/147
Train cost :  -0.554714
Train coef :  0.554714
-----------------
52/1000
90/147
Train cost :  -0.643756
Train coef :  0.643756
-----------------
52/1000
120/147
Train cost :  -0.608538
Train coef :  0.608538
-----------------
53/1000
30/147
Train cost :  -0.625948
Train coef :  0.625948
-----------------
53/1000
60/147
Train cost :  -0.60863
Train coef :  0.60863
-----------------
53/1000
90/147
Train cost :  -0.656585
Train coef :  0.656585
-----------------
53/1000
120/147
Train cost :  -0.406388
Train coef :  0.406388
-----------------
54/1000
30/147
Train cost :  -0.451663
Train coef :  0.451663
-----------------
54/1000
60/147
Train cost :  -0.459408
Train coef :  0.459408
-----------------
54/1000
90/147
Train cost :  -0.57152
Train coef :  0.57152
-----------------
54/1000
120/147
Train cost :  -0.408051
Train coef :  0.408051
-----------------


77/1000
90/147
Train cost :  -0.35761
Train coef :  0.35761
-----------------
77/1000
120/147
Train cost :  -0.502489
Train coef :  0.502489
-----------------
78/1000
30/147
Train cost :  -0.575714
Train coef :  0.575714
-----------------
78/1000
60/147
Train cost :  -0.404513
Train coef :  0.404513
-----------------
78/1000
90/147
Train cost :  -0.618905
Train coef :  0.618905
-----------------
78/1000
120/147
Train cost :  -0.603635
Train coef :  0.603635
-----------------
79/1000
30/147
Train cost :  -0.525811
Train coef :  0.525811
-----------------
79/1000
60/147
Train cost :  -0.535607
Train coef :  0.535607
-----------------
79/1000
90/147
Train cost :  -0.408333
Train coef :  0.408333
-----------------
79/1000
120/147
Train cost :  -0.489693
Train coef :  0.489693
-----------------
80/1000
30/147
Train cost :  -0.504478
Train coef :  0.504478
-----------------
80/1000
60/147
Train cost :  -0.522233
Train coef :  0.522233
-----------------
80/1000
90/147
Train cost :  -0.441971


Train cost :  -0.140023
Train coef :  0.140023
-----------------
103/1000
60/147
Train cost :  -0.747643
Train coef :  0.747643
-----------------
103/1000
90/147
Train cost :  -0.707656
Train coef :  0.707656
-----------------
103/1000
120/147
Train cost :  -0.576613
Train coef :  0.576613
-----------------
104/1000
30/147
Train cost :  -0.682371
Train coef :  0.682371
-----------------
104/1000
60/147
Train cost :  -0.646582
Train coef :  0.646582
-----------------
104/1000
90/147
Train cost :  -0.687361
Train coef :  0.687361
-----------------
104/1000
120/147
Train cost :  -0.656111
Train coef :  0.656111
-----------------
105/1000
30/147
Train cost :  -0.669253
Train coef :  0.669253
-----------------
105/1000
60/147
Train cost :  -0.686835
Train coef :  0.686835
-----------------
105/1000
90/147
Train cost :  -0.687186
Train coef :  0.687186
-----------------
105/1000
120/147
Train cost :  -0.560125
Train coef :  0.560125
-----------------
106/1000
30/147
Train cost :  -0.705222
T

...

Train cost :  -0.545181
Train coef :  0.545181
-----------------
888/1000
30/147
Train cost :  -0.908218
Train coef :  0.908218
-----------------
888/1000
60/147
Train cost :  -0.615194
Train coef :  0.615194
-----------------
888/1000
90/147
Train cost :  -0.961962
Train coef :  0.961962
-----------------
888/1000
120/147
Train cost :  -0.917003
Train coef :  0.917003
-----------------
889/1000
30/147
Train cost :  -0.88903
Train coef :  0.88903
-----------------
889/1000
60/147
Train cost :  -0.927158
Train coef :  0.927158
-----------------
889/1000
90/147
Train cost :  -0.636677
Train coef :  0.636677
-----------------
889/1000
120/147
Train cost :  -0.819301
Train coef :  0.819301
-----------------
890/1000
30/147
Train cost :  -0.547404
Train coef :  0.547404
-----------------
890/1000
60/147
Train cost :  -0.789082
Train coef :  0.789082
-----------------
890/1000
90/147
Train cost :  -0.211194
Train coef :  0.211194
-----------------
890/1000
120/147
Train cost :  -0.534934
Tra

Train coef :  0.884724
-----------------
913/1000
60/147
Train cost :  -0.693916
Train coef :  0.693916
-----------------
913/1000
90/147
Train cost :  -0.505942
Train coef :  0.505942
-----------------
913/1000
120/147
Train cost :  -0.887635
Train coef :  0.887635
-----------------
914/1000
30/147
Train cost :  -0.869097
Train coef :  0.869097
-----------------
914/1000
60/147
Train cost :  -0.900778
Train coef :  0.900778
-----------------
914/1000
90/147
Train cost :  -0.248267
Train coef :  0.248267
-----------------
914/1000
120/147
Train cost :  -0.892289
Train coef :  0.892289
-----------------
915/1000
30/147
Train cost :  -0.850577
Train coef :  0.850577
-----------------
915/1000
60/147
Train cost :  -0.85622
Train coef :  0.85622
-----------------
915/1000
90/147
Train cost :  -0.798808
Train coef :  0.798808
-----------------
915/1000
120/147
Train cost :  -0.900559
Train coef :  0.900559
-----------------
916/1000
30/147
Train cost :  -0.807075
Train coef :  0.807075
----

Train coef :  0.761936
-----------------
938/1000
90/147
Train cost :  -0.910726
Train coef :  0.910726
-----------------
938/1000
120/147
Train cost :  -0.865279
Train coef :  0.865279
-----------------
939/1000
30/147
Train cost :  -0.839043
Train coef :  0.839043
-----------------
939/1000
60/147
Train cost :  -0.886771
Train coef :  0.886771
-----------------
939/1000
90/147
Train cost :  -0.933752
Train coef :  0.933752
-----------------
939/1000
120/147
Train cost :  -0.874336
Train coef :  0.874336
-----------------
940/1000
30/147
Train cost :  -0.437606
Train coef :  0.437606
-----------------
940/1000
60/147
Train cost :  -0.817612
Train coef :  0.817612
-----------------
940/1000
90/147
Train cost :  -0.88926
Train coef :  0.88926
-----------------
940/1000
120/147
Train cost :  -0.416378
Train coef :  0.416378
-----------------
941/1000
30/147
Train cost :  -0.967605
Train coef :  0.967605
-----------------
941/1000
60/147
Train cost :  -0.658337
Train coef :  0.658337
----

963/1000
120/147
Train cost :  -0.946637
Train coef :  0.946637
-----------------
964/1000
30/147
Train cost :  -0.821656
Train coef :  0.821656
-----------------
964/1000
60/147
Train cost :  -0.970667
Train coef :  0.970667
-----------------
964/1000
90/147
Train cost :  -0.877031
Train coef :  0.877031
-----------------
964/1000
120/147
Train cost :  -0.777708
Train coef :  0.777708
-----------------
965/1000
30/147
Train cost :  -0.765862
Train coef :  0.765862
-----------------
965/1000
60/147
Train cost :  -0.797229
Train coef :  0.797229
-----------------
965/1000
90/147
Train cost :  -0.81939
Train coef :  0.81939
-----------------
965/1000
120/147
Train cost :  -0.788391
Train coef :  0.788391
-----------------
966/1000
30/147
Train cost :  -0.962511
Train coef :  0.962511
-----------------
966/1000
60/147
Train cost :  -0.816495
Train coef :  0.816495
-----------------
966/1000
90/147
Train cost :  -0.820593
Train coef :  0.820593
-----------------
966/1000
120/147
Train cost

Train cost :  -0.868738
Train coef :  0.868738
-----------------
989/1000
60/147
Train cost :  -0.971206
Train coef :  0.971206
-----------------
989/1000
90/147
Train cost :  -0.506808
Train coef :  0.506808
-----------------
989/1000
120/147
Train cost :  -0.724895
Train coef :  0.724895
-----------------
990/1000
30/147
Train cost :  -0.762869
Train coef :  0.762869
-----------------
990/1000
60/147
Train cost :  -0.499217
Train coef :  0.499217
-----------------
990/1000
90/147
Train cost :  -0.745069
Train coef :  0.745069
-----------------
990/1000
120/147
Train cost :  -0.904642
Train coef :  0.904642
-----------------
991/1000
30/147
Train cost :  -0.794874
Train coef :  0.794874
-----------------
991/1000
60/147
Train cost :  -0.865891
Train coef :  0.865891
-----------------
991/1000
90/147
Train cost :  -0.619077
Train coef :  0.619077
-----------------
991/1000
120/147
Train cost :  -0.281294
Train coef :  0.281294
-----------------
992/1000
30/147
Train cost :  -0.794862
T

In [61]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    saver = tf.train.Saver()
    saver.restore(sess, './checkpoint/1000')
    test = sess.run(tf.nn.sigmoid(pred['out']), feed_dict={X:np.reshape(tum_in_img[i], [1,256,256,1])})
    
    plt.figure(figsize=(12,12))
    plt.subplot(131)
    plt.xlabel("Ground Truth")
    plt.imshow(tum_in_lab[i, :, :, 0], cmap=plt.cm.bone)
    plt.subplot(132)
    plt.xlabel("Inference")
    plt.imshow(test[0,:,:,0], cmap=plt.cm.bone)
    plt.subplot(133)
    plt.xlabel("Compare")
    plt.imshow(tum_in_lab[i, :, :, 0]-test[0,:,:,0], cmap=plt.cm.bone)
    plt.show()

INFO:tensorflow:Restoring parameters from ./checkpoint/1000


InternalError: Dst tensor is not initialized.
	 [[Node: wd4_3/read/_165 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:GPU:1", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_67_wd4_3/read", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:GPU:1"]()]]
	 [[Node: Sigmoid_234/_193 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_192_Sigmoid_234", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]